# MLOpsへのスコアリングリクエスト

トレーニング環境（[Aqualium](https://aquarium.h2o.ai/login)）Lab 3 MLOps Test Driveに作成済みProject（UCI_Credit_Card）へのHTTPリクエストの実施例

手順：
1. Aqualium Lab3の起動
2. MLOpsへのログイン
3. モデル（UCI_Credit_CardとしてExperimentが実施済み）のデプロイ（No sequlity or Passphrase）
4. デプロイ済みモデルからのサンプルリクエストの取得（Show sample request）

![sample_curl](image/sample_curl.png)

HTTPリクエスト（No sequlity）の実施

```bash
curl -X POST -H "Content-Type: application/json" -d @- http://52.39.41.18:1080/b79a24e5-351b-4390-aca9-7537f4a095a1/model/score << EOF
{
  "fields": ["LIMIT_BAL", "SEX", "EDUCATION", "MARRIAGE", "AGE", "PAY_0", "PAY_2", "PAY_3", "PAY_4", "PAY_5", "PAY_6", "BILL_AMT1", "BILL_AMT2", "BILL_AMT3", "BILL_AMT4", "BILL_AMT5", "PAY_AMT1", "PAY_AMT2", "PAY_AMT3", "PAY_AMT4", "PAY_AMT5", "PAY_AMT6"],
  "rows": [
    ["0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0"]
  ]
}
EOF
```

MLOpsからの応答

```bash
{"fields":["default payment next month.0","default payment next month.1"],"id":"1a6280c6-1f71-11eb-a0a1-0242ac11000f","score":[["0.6314364746212959","0.36856352537870407"]]}
```

## Python ([Request](https://requests-docs-ja.readthedocs.io/en/latest/#))での実施

In [5]:
import requests
import json
import pandas as pd

In [24]:
# APIポイント
api_point = "http://54.185.209.232:1080/236bbf9b-ad2b-4f9f-b7bc-bb8117aff854/model/score"
# header
headers = {'Content-Type': 'application/json'}

In [25]:
# カラム定義
columns = ["LIMIT_BAL", "SEX", "EDUCATION", "MARRIAGE", "AGE", "PAY_0", "PAY_2", "PAY_3", "PAY_4", "PAY_5", "PAY_6", "BILL_AMT1", "BILL_AMT2", "BILL_AMT3", "BILL_AMT4", "BILL_AMT5", "PAY_AMT1", "PAY_AMT2", "PAY_AMT3", "PAY_AMT4", "PAY_AMT5", "PAY_AMT6"]

In [99]:
## 数値Listの文字列化

def elem_to_str(lst):
    ''' [1,2,3]  ->  ['1','2','3']
    '''
    return [str(elem) for elem in lst]

def list_to_str(lst_external):
    ''' [[1,2,3], [4,5,6]]  ->  [['1','2','3'], ['4','5','6']]
    '''
    lst_return = []
    for lst in lst_external:
        lst_return.append(elem_to_str(lst))
    return lst_return

In [128]:
## pred(y=1)をListで返す

def get_pred(res_lst):
    proba_list = []
    for lst in res_lst:
        proba_list.append(float(lst[1]))
    return proba_list

### 1行スコアリング

In [129]:
data = [[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]
list_to_str(data)

[['0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0',
  '0']]

In [130]:
json_data = {"fields":columns, "rows":list_to_str(data)}
json_data

{'fields': ['LIMIT_BAL',
  'SEX',
  'EDUCATION',
  'MARRIAGE',
  'AGE',
  'PAY_0',
  'PAY_2',
  'PAY_3',
  'PAY_4',
  'PAY_5',
  'PAY_6',
  'BILL_AMT1',
  'BILL_AMT2',
  'BILL_AMT3',
  'BILL_AMT4',
  'BILL_AMT5',
  'PAY_AMT1',
  'PAY_AMT2',
  'PAY_AMT3',
  'PAY_AMT4',
  'PAY_AMT5',
  'PAY_AMT6'],
 'rows': [['0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0']]}

In [131]:
# POSTするデータ
json.dumps(json_data)

'{"fields": ["LIMIT_BAL", "SEX", "EDUCATION", "MARRIAGE", "AGE", "PAY_0", "PAY_2", "PAY_3", "PAY_4", "PAY_5", "PAY_6", "BILL_AMT1", "BILL_AMT2", "BILL_AMT3", "BILL_AMT4", "BILL_AMT5", "PAY_AMT1", "PAY_AMT2", "PAY_AMT3", "PAY_AMT4", "PAY_AMT5", "PAY_AMT6"], "rows": [["0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0"]]}'

In [132]:
response = requests.post(url=api_point, headers=headers, data=json.dumps(json_data))
response

<Response [200]>

In [133]:
response.json()

{'fields': ['default payment next month.0', 'default payment next month.1'],
 'id': '1a6280c6-1f71-11eb-a0a1-0242ac11000f',
 'score': [['0.6314364746212959', '0.36856352537870407']]}

In [136]:
# prob(y=1)の取得
get_pred(response.json()['score'])

[0.36856352537870407]

### 複数行スコアリング

In [137]:
data = [[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0], 
              [90000,2,2,2,34,0,0,0,0,0,0,29239,14027,13559,14331,14948,1518,1500,1000,1000,1000,5000],
              [50000,2,2,1,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,2000,2019,1200,1100,1069,1000]]

json_data = {"fields":columns, "rows":to_str_list(data)}
response = requests.post(url=api_point, headers=headers, data=json.dumps(json_data))
response.json()

{'fields': ['default payment next month.0', 'default payment next month.1'],
 'id': '1a6280c6-1f71-11eb-a0a1-0242ac11000f',
 'score': [['0.6314364746212959', '0.36856352537870407'],
  ['0.901497257873416', '0.09850274212658405'],
  ['0.8544589467346668', '0.14554105326533318']]}

In [138]:
get_pred(response.json()['score'])

[0.36856352537870407, 0.09850274212658405, 0.14554105326533318]

### csvからのスコアリング

In [141]:
df = pd.read_csv('sample_scoring_UciCreditCard.csv')
print(df.shape)
df

(7, 22)


,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
0,20000,2,2,1,24,2,2,-1,-1,-2,...,3102,689,0,0,0,689,0,0,0,0
1,120000,2,2,2,26,-1,2,0,0,0,...,1725,2682,3272,3455,0,1000,1000,1000,0,2000
2,90000,2,2,2,34,0,0,0,0,0,...,14027,13559,14331,14948,1518,1500,1000,1000,1000,5000
3,50000,2,2,1,37,0,0,0,0,0,...,48233,49291,28314,28959,2000,2019,1200,1100,1069,1000
4,50000,1,2,1,57,-1,0,-1,0,0,...,5670,35835,20940,19146,2000,36681,10000,9000,689,679
5,50000,1,1,2,37,0,0,0,0,0,...,57069,57608,19394,19619,2500,1815,657,1000,1000,800
6,500000,1,1,2,29,0,0,0,0,0,...,412023,445007,542653,483003,55000,40000,38000,20239,13750,13770


In [162]:
# データ型の変換
df_str = df.astype(str)
df_str.dtypes

LIMIT_BAL    object
SEX          object
EDUCATION    object
MARRIAGE     object
AGE          object
PAY_0        object
PAY_2        object
PAY_3        object
PAY_4        object
PAY_5        object
PAY_6        object
BILL_AMT1    object
BILL_AMT2    object
BILL_AMT3    object
BILL_AMT4    object
BILL_AMT5    object
PAY_AMT1     object
PAY_AMT2     object
PAY_AMT3     object
PAY_AMT4     object
PAY_AMT5     object
PAY_AMT6     object
dtype: object

In [163]:
# pandas.DataFrameから、jsonに変換
json_data = df_str.to_json(orient="split")
json_data = json.loads(json_data)
json_data

{'columns': ['LIMIT_BAL',
  'SEX',
  'EDUCATION',
  'MARRIAGE',
  'AGE',
  'PAY_0',
  'PAY_2',
  'PAY_3',
  'PAY_4',
  'PAY_5',
  'PAY_6',
  'BILL_AMT1',
  'BILL_AMT2',
  'BILL_AMT3',
  'BILL_AMT4',
  'BILL_AMT5',
  'PAY_AMT1',
  'PAY_AMT2',
  'PAY_AMT3',
  'PAY_AMT4',
  'PAY_AMT5',
  'PAY_AMT6'],
 'index': [0, 1, 2, 3, 4, 5, 6],
 'data': [['20000',
   '2',
   '2',
   '1',
   '24',
   '2',
   '2',
   '-1',
   '-1',
   '-2',
   '-2',
   '3913',
   '3102',
   '689',
   '0',
   '0',
   '0',
   '689',
   '0',
   '0',
   '0',
   '0'],
  ['120000',
   '2',
   '2',
   '2',
   '26',
   '-1',
   '2',
   '0',
   '0',
   '0',
   '2',
   '2682',
   '1725',
   '2682',
   '3272',
   '3455',
   '0',
   '1000',
   '1000',
   '1000',
   '0',
   '2000'],
  ['90000',
   '2',
   '2',
   '2',
   '34',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '29239',
   '14027',
   '13559',
   '14331',
   '14948',
   '1518',
   '1500',
   '1000',
   '1000',
   '1000',
   '5000'],
  ['50000',
   '2',
   '2',
   '1

In [164]:
# キー名の変更
json_data['fields'] = json_data['columns']
del json_data['columns']
json_data['rows'] = json_data['data']
del json_data['data']

In [166]:
response = requests.post(url=api_point, headers=headers, data=json.dumps(json_data))
response.json()

{'fields': ['default payment next month.0', 'default payment next month.1'],
 'id': '1a6280c6-1f71-11eb-a0a1-0242ac11000f',
 'score': [['0.3035139739513397', '0.6964860260486603'],
  ['0.6079196035861969', '0.3920803964138031'],
  ['0.901497257873416', '0.09850274212658405'],
  ['0.8544589467346668', '0.14554105326533318'],
  ['0.89368318580091', '0.10631681419909'],
  ['0.856270395219326', '0.14372960478067398'],
  ['0.8821506686508656', '0.11784933134913445']]}

In [167]:
get_pred(response.json()['score'])

[0.6964860260486603,
 0.3920803964138031,
 0.09850274212658405,
 0.14554105326533318,
 0.10631681419909,
 0.14372960478067398,
 0.11784933134913445]

### Passphraseを指定した場合
デプロイの際のEndpoint Security Levelを、"Passphrase (scored as plain text)"もしくは"Passphrase (stored hashed)"を選択

![sample_curl_pass](image/sample_curl_pass.png)

```bash
curl -X POST -H "Content-Type: application/json" -H "Authorization: Bearer mypassword" -d @- http://54.185.209.232:1080/efddad09-91d2-4f3e-a51f-dd95650fe63b/model/score << EOF
{
  "fields": ["LIMIT_BAL", "SEX", "EDUCATION", "MARRIAGE", "AGE", "PAY_0", "PAY_2", "PAY_3", "PAY_4", "PAY_5", "PAY_6", "BILL_AMT1", "BILL_AMT2", "BILL_AMT3", "BILL_AMT4", "BILL_AMT5", "PAY_AMT1", "PAY_AMT2", "PAY_AMT3", "PAY_AMT4", "PAY_AMT5", "PAY_AMT6"],
  "rows": [
    ["0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0"]
  ]
}
EOF
```

In [178]:
# APIポイント
api_point = "http://54.185.209.232:1080/efddad09-91d2-4f3e-a51f-dd95650fe63b/model/score"
# header
headers = {'Content-Type': 'application/json', 'Authorization': 'Bearer mypassword'}

In [179]:
data = [[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]
json_data = {"fields":columns, "rows":to_str_list(data)}
response = requests.post(url=api_point, headers=headers, data=json.dumps(json_data))
response.json()

{'fields': ['default payment next month.0', 'default payment next month.1'],
 'id': '1a6280c6-1f71-11eb-a0a1-0242ac11000f',
 'score': [['0.6314364746212959', '0.36856352537870407']]}